In [1]:
import json
import pandas as pd

In [2]:
eval_path = "log-output/2024-11-23T00-27-58-08-00_theory-of-mind_fZviDx4z7a8UaPgMmCNmpD.json"
with open(eval_path, "r") as f:
    eval_data = dict(json.load(f))

eval_data['eval'].keys()



dict_keys(['run_id', 'created', 'task', 'task_id', 'task_version', 'task_file', 'task_attribs', 'task_args', 'dataset', 'model', 'model_args', 'config', 'revision', 'packages'])

In [36]:
samples = eval_data['eval']['dataset']['samples']
model = eval_data['eval']['model']
prompt = eval_data['plan']['steps'][0]['params']['template']
accuracy = eval_data['results']['scores'][0]['metrics']['accuracy']['value']
stderr = eval_data['results']['scores'][0]['metrics']['stderr']['value']
run_start = eval_data['stats']['started_at']
run_end = eval_data['stats']['completed_at']
run_input_tokens = eval_data['stats']['model_usage'][model]['input_tokens']
run_output_tokens = eval_data['stats']['model_usage'][model]['output_tokens']
run_total_tokens = eval_data['stats']['model_usage'][model]['total_tokens']
run_id = eval_data['eval']['run_id']
run_data = {
    'run_id': run_id,
    'model': model,
    'prompt': prompt,
    'accuracy': accuracy,
    'stderr': stderr,
    'run_start': run_start,
    'run_end': run_end,
}

run_data_df = pd.DataFrame([run_data])
run_data_df['upper_bound'] = run_data_df['accuracy'] + 1.96 * run_data_df['stderr']
run_data_df['lower_bound'] = run_data_df['accuracy'] - 1.96 * run_data_df['stderr']
run_data_df.to_json('run_data.json', orient='records')



In [39]:
samples = eval_data['samples']
len(samples)

100

In [40]:

final_samples = []

for sample in samples:   
    
    sample_id = sample['id']
    epoch = sample['epoch']
    input = sample['input'][0]['content']
    target = sample['target']
    messages = sample['messages']
    output = sample['output']['choices'][0]['message']['content']
    score = sample['scores']['model_graded_fact']['value']

    final_samples.append({
        'run_id': run_id,
        'sample_id': sample_id,
        'epoch': epoch,
        'input': input,
        'target': target,
        'output': output,
        'score': score,
    })


run_data_samples_df = pd.DataFrame(final_samples)
run_data_samples_df['score_binary'] = run_data_samples_df['score'].apply(lambda x: 1 if x == 'C' else 0)
run_data_samples_df['cumulative_score'] = run_data_samples_df['score_binary'].cumsum()
run_data_samples_df['cumulative_score_percentage'] = run_data_samples_df['cumulative_score'] / run_data_samples_df['sample_id']
run_data_samples_df.to_json('run_data_samples.json', orient='records')

score_binary
1    79
0    21
Name: count, dtype: int64